In [ ]:
### get image file names (subject ID + '.npy') as list
base_dir = '/home/connectome/dhkdgmlghks/3DCNN_test'
data_dir = '/home/connectome/dhkdgmlghks/3DCNN_test/preprocessed_masked'
os.chdir(data_dir)
image_files = glob.glob('*.npy')
image_files = sorted(image_files)
#image_files = image_files[:100]
print("Loading image file names as list is completed")
## ====================================== ##

## ========= Data Preprocessing: target ========= ##
### get subject ID and target variables
target = 'SuicidalideationPassive.x'

subject_dir = '/home/connectome/dhkdgmlghks/3DCNN_test'
os.chdir(subject_dir)

control_data = pd.read_csv('ABCD_suicide_control.csv')
control_data = control_data.loc[:,['subjectkey',target]]
control_data = control_data.sort_values(by='subjectkey')
control_data = control_data.dropna(axis = 0)
control_data = control_data.reset_index(drop=True) # removing subject have NA values in sex

case_data = pd.read_csv('ABCD_suicide_case.csv')
case_data = case_data.loc[:,['subjectkey',target]]
case_data = case_data.sort_values(by='subjectkey')
case_data = case_data.dropna(axis = 0)
case_data = case_data.reset_index(drop=True) # removing subject have NA values in sex

print("Loading subject list is completed")
## ====================================== ##

## ========= Data Preprocessing ========= ##
os.chdir(data_dir) # if I do not change directory here, image data is not loaded
# get subject ID and target variables as sorted list

imageFiles_labels_control = []
for subjectID in tqdm(image_files):
    subjectID = subjectID[:-4] #removing '.npy' for comparing
    #print(subjectID)
    for i in range(len(control_data)):
        if subjectID == control_data['subjectkey'][i]:
            if control_data[target][i] == 0:
                imageFiles_labels_control.append((subjectID+'.npy',0))
            elif control_data[target][i] == 1:
                imageFiles_labels_control.append((subjectID+'.npy',1))
            else:
                print('NaN value for {}'.format(subjectID))
                continue

imageFiles_labels_case = []
for subjectID in tqdm(image_files):
    subjectID = subjectID[:-4] #removing '.npy' for comparing
    #print(subjectID)
    for i in range(len(case_data)):
        if subjectID == case_data['subjectkey'][i]:
            if case_data[target][i] == 0:
                imageFiles_labels_case.append((subjectID+'.npy',0))
            elif case_data[target][i] == 1:
                imageFiles_labels_case.append((subjectID+'.npy',1))
            else:
                print('NaN value for {}'.format(subjectID))
                continue

imageFiles_labels = {}
imageFiles_labels['control'] = imageFiles_labels_control
imageFiles_labels['case'] = imageFiles_labels_case
## ====================================== ##

## ========= Split train, val, test========= ##
# defining train,val, test set splitting function
def partition(imageFiles_labels,args):
    #random.shuffle(imageFiles_labels)

    images_control = []
    labels_control = []
    for imageFile_label in imageFiles_labels['control']:
        image, label = imageFile_label
        images_control.append(image)
        labels_control.append(label)

    images_case = []
    labels_case = []
    for imageFile_label in imageFiles_labels['case']:
        image, label = imageFile_label
        images_case.append(image)
        labels_case.append(label)


    resize = args.resize
    train_transform = Compose([ScaleIntensity(),
                               AddChannel(),
                               Resize(resize),
                              ToTensor()])

    val_transform = Compose([ScaleIntensity(),
                               AddChannel(),
                               Resize(resize),
                              ToTensor()])

    test_transform = Compose([ScaleIntensity(),
                               AddChannel(),
                               Resize(resize),
                              ToTensor()])


    # making balanced case/control test set
    case_total = len(images_case)
    case_test = int(case_total*args.test_size)
    case_val = int(case_total*args.val_size)
    
    images_test_case = images_case[:case_test]
    labels_test_case = labels_case[:case_test]

    images_val_case = images_case[case_test:case_test+case_val]
    labels_val_case = labels_case[case_test:case_test+case_val]     
    
    images_train_case = images_case[case_test+case_val:]
    labels_train_case = labels_case[case_test+case_val:] 


    control_total = len(images_control)
    control_test = case_test
    control_val = case_val
    
    images_test_control = images_control[:control_test]
    labels_test_control = labels_control[:control_test]

    images_val_control = images_control[control_test:control_test+control_val]
    labels_val_control = labels_control[control_test:control_test+control_val]     
    
    images_train_control = images_case[case_test+case_val:]
    labels_train_control = labels_case[case_test+case_val:] 


    images_train = images_train_case + images_train_control
    images_val = images_val_case + images_val_control
    images_test = images_test_case + images_test_control

    labels_train = labels_train_case + labels_train_control
    labels_val = labels_val_case + labels_val_control
    labels_test = labels_test_case + labels_test_control
     

    train_set = ImageDataset(image_files=images_train,labels=labels_train,transform=train_transform)
    val_set = ImageDataset(image_files=images_val,labels=labels_val,transform=val_transform)
    test_set = ImageDataset(image_files=images_test,labels=labels_test,transform=test_transform)

    partition = {}
    partition['train'] = train_set
    partition['val'] = val_set
    partition['test'] = test_set

    return partition

# split dataset
partition = partition(imageFiles_labels,args)
print("Splitting data to train, val, test is completed")